In [3]:
import ee
import zipfile
import geopandas as gpd
import pandas as pd
import sys
import os
import tempfile
import datetime
import requests

In [4]:
ee.Authenticate()
ee.Initialize(project='farm-1111')

In [5]:
shapefile_path = "./shape/contorno_area_total.shp"

# Load shapefile (supports both .shp and .zip formats)
if shapefile_path.endswith('.zip'):
    aoi = gpd.read_file(f'zip://{shapefile_path}')
else:
    aoi = gpd.read_file(shapefile_path)

# Dissolve geometries if multiple exist
aoi = aoi.dissolve()

geometry = aoi.geometry.iloc[0]

# Convert to GeoJSON and remove third dimension
geojson = geometry.__geo_interface__
geojson['coordinates'] = [
    [coord[:2] for coord in ring] if geojson['type'] == 'Polygon' else
    [[coord[:2] for coord in ring] for ring in polygon]
    for ring in geojson['coordinates']
]

# Create Earth Engine FeatureCollection
aoi = ee.FeatureCollection([ee.Feature(ee.Geometry(geojson))])

In [7]:
dataset_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
#.filterDate(start, end) \
.filterBounds(aoi2) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
#.filter(ee.Filter.lt('WATER_PERCENTAGE', 2)) \
.map(maskS2clouds) \
# .map(lambda image: image.clip(aoi2)) \
# .select(s2_names, b_names)


SyntaxError: invalid syntax (2424434717.py, line 3)

In [ ]:
# --- Definição da Área de Interesse ---
aoi2 = aoi.geometry().bounds()

# --- Definição de Variáveis e Listas ---
s2_names = ['B2', 'B3', 'B4', 'B6', 'B8','B11','B12','QA60']
b_names = ['blue', 'green', 'red','rededge', 'nir', 'swir1', 'swir2', 'QA60']

# --- Funções de Processamento ---

def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 e 11 são nuvens e cirrus
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Ambos devem ser zero
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

def add_indexes(img):
    ndvi = img.normalizedDifference(['nir', 'red']) 
    nbr2 = img.normalizedDifference(['swir1', 'swir2'])
    grbl = img.select('green').subtract(img.select('blue'))
    regr = img.select('red').subtract(img.select('green'))
    
    img = img.addBands(ndvi.rename('ndvi')) \
             .addBands(nbr2.rename('nbr2')) \
             .addBands(grbl.rename('grbl')) \
             .addBands(regr.rename('regr'))
    return img

def applyScaleFactors(image):
    opticalBands = image.select(b_names).divide(10000)
    return image.addBands(opticalBands, None, True)

def addGEOS3Mask(img, options):
    # Extração de opções do dicionário
    rescale_flag = options.get('rescale_flag', 0)
    ndvi_thres = options.get('ndvi_thres', [-0.25, 0.25])
    nbr_thres = options.get('nbr_thres', [-0.3, 0.1])
    vnsir_thres = options.get('vnsir_thres', 0.9)

    if rescale_flag == 1:
        img = img.divide(10000)

    # Índice de tendência Visible-to-shortwave-infrared
    vnsir = ee.Image(1).subtract(
        ee.Image(2).multiply(img.select('red'))
        .subtract(img.select('green')).subtract(img.select('blue'))
        .add(ee.Image(3).multiply(img.select('nir').subtract(img.select('red'))))
    )
              
    # Lógica GEOS3
    geos3 = img.select('ndvi').gte(ndvi_thres[0]).And(img.select('ndvi').lte(ndvi_thres[1])) \
        .And(img.select('nbr2').gte(nbr_thres[0]).And(img.select('nbr2').lte(nbr_thres[1]))) \
        .And(vnsir.lte(vnsir_thres)) \
        .And(img.select('grbl').gt(0)).And(img.select('regr').gt(0))
              
    img = img.addBands(geos3.rename('GEOS3'))
    return img

def maskByGEOS3(image):
    # Cria máscara onde GEOS3 é verdadeiro (1)
    mask_geos3 = image.select('GEOS3').eq(1)
    # Removendo linhas pretas/bordas
    mask_swir = image.select('swir2').gte(0)
    mask_green = image.select('green').gte(0)
    mask_red = image.select('red').gte(0)
    mask_blue = image.select('blue').gte(0)
    
    mask = mask_geos3.And(mask_swir).And(mask_green).And(mask_red).And(mask_blue)
    return image.updateMask(mask)

def calculateMean(image, region):
    return image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=30,
        maxPixels=1e8
    )

# --- Fluxo Principal de Execução ---

# 1. Carregar Coleção e Filtragem Inicial
dataset_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi2) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .filter(ee.Filter.lt('WATER_PERCENTAGE', 2)) \
    .map(maskS2clouds) \
    .map(lambda image: image.clip(aoi2)) \
    .select(s2_names, b_names)

# 2. Adicionar Índices e Fatores de Escala
s2_indexes = dataset_s2.map(add_indexes)
s2_indexes = s2_indexes.map(applyScaleFactors)

# 4. Aplicação da Máscara GEOS3 na coleção
# Opções fixas conforme o script original
geos3_options = {
    'rescale_flag': 0, 
    'ndvi_thres': [-0.25, 0.25], 
    'nbr_thres': [-0.3, 0.100], 
    'vnsir_thres': 0.9
}

sent_collection_geos3 = s2_indexes.map(lambda img: addGEOS3Mask(img, geos3_options))

# 5. Obter a mediana dos pixels (TESS)
tess_v1_collection = sent_collection_geos3.map(maskByGEOS3)
tess_v1 = tess_v1_collection.median()


In [ ]:
dataset_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi2) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .filter(ee.Filter.lt('WATER_PERCENTAGE', 2)) \
    .map(maskS2clouds) \
    .map(lambda image: image.clip(aoi2)) \
    .select(s2_names, b_names)

In [5]:
final_image = tess_v1.toFloat()

# --- KEY CHANGE: Use updateMask for more reliable clipping ---
# Apply buffer if needed. Ensure self.aoi is an ee.Geometry object.

# 1. Create an explicit mask from the AOI geometry.
# ee.Image(1).clip(aoi) creates an image where the AOI is 1 and outside is 0.
# .mask() converts this to a mask where 1 is valid data and 0 is NoData.
mask = ee.Image(1).clip(aoi).mask()

# 2. Apply this mask to the final composite image.
# This operation sets pixels outside the mask to NoData (transparent).
final_image_masked = final_image.updateMask(mask)

# 3. Define the download region using the BOUNDING BOX of the AOI.
# This ensures the downloaded GeoTIFF is a rectangle that fully covers the AOI.
# The actual clipping to the irregular shape is handled by updateMask.
download_region = aoi.geometry().bounds().getInfo()

In [6]:
# Download the selected composite (all selected bands) and load as a raster layer

url = final_image_masked.getDownloadURL(
    {
        "scale": 10,
        "region": download_region,
        "format": "GeoTIFF",
        "crs": "EPSG:4326",
    }
)

output_file = "teste.tiff"


response = requests.get(url, stream=True)
response.raise_for_status()
with open(output_file, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)
print(f"Image downloaded to {output_file}")


Image downloaded to teste.tiff
